<a href="https://colab.research.google.com/github/haleemaraza/agent_sdk_guradrails_7/blob/main/guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.5/168.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 9.2 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pydantic import BaseModel
from agents import (
    Agent,
    RunContextWrapper,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    TResponseInputItem,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    input_guardrail,
    output_guardrail
)
from google.colab import userdata

ImportError: cannot import name 'ResponseTextConfigParam' from 'openai.types.responses' (/usr/local/lib/python3.12/dist-packages/openai/types/responses/__init__.py)

In [ ]:
google_api_key = userdata.get("GOOGLE_API_KEY")

if not google_api_key:
  raise ValueError("GOOGLE_PI_KEY is not set . please ensure it is defined in your .env. file")


In [ ]:
external_client=AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",
)

model= OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [ ]:
class MathHomeworkOutput(BaseModel):
  is_math_homework:bool
  reasoning:str
  answer:str

guardrail_agent= Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework",
    output_type=MathHomeworkOutput,
    model=model
)

In [ ]:
output= Runner.run_sync(guardrail_agent, "what is the capital of pakistan?")

print("is_math_homework: \n", output.final_output.is_math_homework)
print("reasoning: \n",output.final_output.reasoning)
print("answer: \n", output.final_output.answer)

In [ ]:
output = Runner.run_sync(guardrail_agent, "what is factorial of 2")
output.final_output

In [ ]:
@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent:Agent, input:str|list[TResponseInputItem]) -> GuardrailFunctionOutput:
    result=await Runner.run(guardrail_agent, input, context=ctx.context)

    print("\n\n [GUARDRAIL_RESPONSE]", result.final_output,"\n\n")

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math_homework,
    )

In [ ]:
agent=Agent(
    name="Math Homework Assistant",
    instructions="you are a math support agent you help student with their questions.",
    input_guardrails=[math_guardrail],
    model=model
)